In [1]:
import json
import pandas as pd
from pathlib import Path
from pandas import json_normalize
from tqdm import tqdm

In [2]:
assessment_data = Path(f'out/assessment_data')
assessment_data.exists()

True

In [3]:
assessment_dir_paths = list(assessment_data.glob('*/'))
assessment_dir_paths = list(filter(lambda x: '.DS_Store' not in x.name, assessment_dir_paths))
assessment_dir_paths

[PosixPath('out/assessment_data/100-trials_conf-geq-70_sample-from-val'),
 PosixPath('out/assessment_data/100-trials_conf-geq-70_sample-from-dswx')]

In [4]:
def get_dswx_ids(assessment_dir) -> list:
    dswx_verification_paths = list(assessment_dir.glob('*/'))
    dswx_ids = [path.name for path in dswx_verification_paths]
    # Remove DSWx paths
    dswx_ids = list(filter(lambda dswx_id: '.' != dswx_id[0], dswx_ids))
    return dswx_ids


def combine_requirement_verification_for_one_assessment(assessment_dir_path):
    out_dir = Path(f'out/verification_stats_agg/{assessment_dir_path.name}')
    out_dir.mkdir(exist_ok=True, parents=True)
    
    dswx_ids = get_dswx_ids(assessment_dir_path)
    
    def read_one_requirements_json(dswx_id):
        path = assessment_dir_path / dswx_id / f'requirement_verification_{dswx_id}.json'
        data = json.load(open(path))
        return data
    
    data = list(map(read_one_requirements_json, dswx_ids))
    df = pd.DataFrame(data)
    cols = ['surface_water','partial_surface_water']
    df[cols + ['dswx_id']].groupby(cols).count()
    
    out_path = out_dir / 'verification_results.csv'
    df.to_csv(out_path, index=False)
    
    return df, out_path

In [5]:
dfs_req, out_paths = zip(*list(map(combine_requirement_verification_for_one_assessment, tqdm(assessment_dir_paths))))

100%|███████████████████████████████████| 2/2 [00:00<00:00, 75.96it/s]


In [6]:
dfs_req[0].head()

,dswx_id,surface_water,partial_surface_water,dswx-WTR,dswx-BWTR,dswx-CONF,dswx-DIAG,dswx-WTR-1,dswx-WTR-2,dswx-LAND,...,hls_url_B04,hls_url_B05,hls_url_B06,hls_url_B07,hls_url_B08,hls_url_B09,hls_url_B10,hls_url_B11,hls_url_B12,hls_url_Fmask
0,OPERA_L3_DSWx-HLS_T19HCT_20211027T142729Z_2023...,True,True,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
1,OPERA_L3_DSWx-HLS_T55TCJ_20210916T011649Z_2023...,True,True,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
2,OPERA_L3_DSWx-HLS_T21JWF_20211028T133741Z_2023...,True,True,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
3,OPERA_L3_DSWx-HLS_T20LKN_20211010T143729Z_2023...,False,False,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
4,OPERA_L3_DSWx-HLS_T12SXH_20210921T175632Z_2023...,True,True,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3

In [7]:
out_paths[0]

PosixPath('out/verification_stats_agg/100-trials_conf-geq-70_sample-from-val/verification_results.csv')

# Additional Metrics

In [8]:
def combine_metrics_for_one_assessment(assessment_dir_path):
    out_dir = Path(f'out/verification_stats_agg/{assessment_dir_path.name}')
    
    def read_one_assessment(dswx_id):
        path = assessment_dir_path / dswx_id / f'stats_{dswx_id}.json'
        data = pd.DataFrame(pd.json_normalize(json.load(open(path))))
        return data
    
    dswx_ids = get_dswx_ids(assessment_dir_path)
    
    data_assessments = list(map(read_one_assessment, dswx_ids))
    df_metrics = pd.concat(data_assessments, axis=0).reset_index(drop=True)
    df_metrics.to_csv(out_dir / 'metrics.csv', index=False)
    out_path = out_dir / 'metrics.csv'
    df_metrics.to_csv(out_path, index=False)
    return df_metrics, out_path

In [9]:
dfs_metrics, out_paths = zip(*list(map(combine_metrics_for_one_assessment, tqdm(assessment_dir_paths))))

100%|███████████████████████████████████| 2/2 [00:00<00:00, 26.89it/s]


In [10]:
dfs_metrics[0].head()

,total_accuracy.mean,total_accuracy.std,binary_water_acc.mean,binary_water_acc.std,acc_per_class.Not_Water.mean,acc_per_class.Not_Water.std,acc_per_class.Open_Surface_Water.mean,acc_per_class.Open_Surface_Water.std,acc_per_class.Partial_Surface_Water.mean,acc_per_class.Partial_Surface_Water.std,...,f1_per_class.Partial_Surface_Water.mean,f1_per_class.Partial_Surface_Water.std,supp_per_class.Not_Water.mean,supp_per_class.Not_Water.std,supp_per_class.Open_Surface_Water.mean,supp_per_class.Open_Surface_Water.std,supp_per_class.Partial_Surface_Water.mean,supp_per_class.Partial_Surface_Water.std,dswx_id,planet_id
0,0.825661,0.005797,0.885754,0.004014,0.885754,0.004014,0.923364,0.005620,0.842204,0.004331,...,0.527829,0.006846,167.0,0.0,167.0,0.0,97.0,0.0,OPERA_L3_DSWx-HLS_T19HCT_20211027T142729Z_2023...,20211028_144231_39_227b
1,0.704212,0.006329,0.882974,0.009669,0.882974,0.009669,0.820719,0.009107,0.704731,0.006262,...,0.205881,0.029401,167.0,0.0,167.0,0.0,167.0,0.0,OPERA_L3_DSWx-HLS_T55TCJ_20210916T011649Z_2023...,20210916_010848_94_2407
2,0.932836,0.000000,0.932836,0.000000,0.932836,0.000000,1.000000,0.000000,0.932836,0.000000,...,0.000000,0.000000,250.0,0.0,0.0,0.0,18.0,0.0,OPERA_L3_DSWx-HLS_T21JWF_20211028T133741Z_2023...,20211028_134803_20_227a
3,0.675165,0.005819,0.900207,0.005637,0.900207,0.005637,0.774959,0.001896,0.675165,0.005819,...,0.593442,0.010207,167.0,0.0,150.0,0.0,167.0,0.0,OPERA_L3_DSWx-HLS_T20LKN_20211010T143729Z_2023...,20211010_135831_84_227e
4,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,500.0,0.0,0.0,0.0,0.0,0.0,OPERA_L3_DSWx-HLS_T12SXH_20210921T175632Z_2023...,20210922_171337_39_2420


In [11]:
out_paths[0]

PosixPath('out/verification_stats_agg/100-trials_conf-geq-70_sample-from-val/metrics.csv')

# For presentations

Aggregate all validation statistics for a particular set of validation runs

In [12]:
index = [k for k, out_path in enumerate(out_paths) if '100-trials_conf-geq-70_sample-from-val' in str(out_path)][0]
index

0

In [13]:
df_final = dfs_metrics[index]
out_path_final = out_paths[index]

In [14]:
COLUMN_RENAME = {'total_accuracy.mean': 'Total Accuracy (All Classes)',
                 'binary_water_acc.mean': 'Binary Accuarcy (OSW + PSW)',
                 'precision.Open_Surface_Water.mean': 'Precision (OSW)',
                 'acc_per_class.Open_Surface_Water.mean': 'Binary Accuracy (OSW)',
                 'acc_per_class.Partial_Surface_Water.mean': 'Binary Accuracy (PSW)',
                 'precision.Partial_Surface_Water.mean': 'Precision (PSW)',
                 'recall.Open_Surface_Water.mean': 'Recall (OSW)',
                 'recall.Partial_Surface_Water.mean': 'Recall (PSW)',
                 #'supp_per_class.Open_Surface_Water.mean': 'Support (Val) (OSW)',
                 #'supp_per_class.Partial_Surface_Water.mean': 'Support (Val) (PSW)',
                 'f1_per_class.Open_Surface_Water.mean': 'F1 (OSW)',
                 'f1_per_class.Partial_Surface_Water.mean': 'F1 (PSW)'}
COLUMNS = list(COLUMN_RENAME.keys())

In [15]:
df_temp = df_final[COLUMNS].agg(['mean', 'median', 'std'])
df_temp.rename(columns=COLUMN_RENAME, inplace=True)

cols_not_supp = [col for col in df_temp.columns if 'Support' not in col]
df_temp[cols_not_supp] = df_temp[cols_not_supp] * 100

df_f = df_temp.T
df_f = df_f.round(2).astype(str)

df_f = df_f.reset_index(drop=False)
df_f = df_f.rename(columns={'index': 'Metric'})
def class_labeler(metric):
    if '(OSW + PSW)' in metric:
        return 'OSW + PSW'
    if 'OSW' in metric:
        return 'OSW'
    if 'PSW' in metric:
        return 'PSW'
    else:
        return 'All'
df_f['Class'] = df_f.Metric.map(class_labeler)

lookup_order = {'All': 0, 'OSW + PSW': 1, 'OSW': 2, 'PSW': 3}
df_f['Class_sort'] = df_f.Class.map(lambda c: lookup_order[c])

df_f['Metric'] = df_f.Metric.map(lambda m: m.split('(')[0])
df_f['Metric'] = df_f.Metric.map(lambda m: m + ' ($\%$)' if 'Support' not in m else m)
df_f.sort_values(by=['Class_sort', 'Metric'], inplace=True)
df_f = df_f.set_index(['Class', 'Metric'])
df_f.drop(columns=['Class_sort'], inplace=True)
df_f.rename(columns={'mean': 'Mean', 
                     'std': 'St. Dev.', 
                     'median': 'Median'}, inplace=True)
df_f

Mean Median St. Dev.
Class     Metric                                        
All       Total Accuracy  ($\%$)   84.18  89.51     17.8
OSW + PSW Binary Accuarcy  ($\%$)  90.86   98.0    13.32
OSW       Binary Accuracy  ($\%$)  91.63  97.89    10.85
          F1  ($\%$)               80.58  92.96    28.57
          Precision  ($\%$)        91.27  100.0    14.59
          Recall  ($\%$)           80.79  99.75     31.6
PSW       Binary Accuracy  ($\%$)  85.88  89.62     14.7
          F1  ($\%$)               62.77  61.65    35.92
          Precision  ($\%$)        78.41  89.81    27.89
          Recall  ($\%$)           63.08  73.15    38.12

In [16]:
presentation_dir = Path('presentation_images') / out_path_final.parent.name 
presentation_dir.mkdir(exist_ok=True, parents=True)
presentation_dir

PosixPath('presentation_images/100-trials_conf-geq-70_sample-from-val')

In [17]:
latex = df_f.style.to_latex(multirow_align='t', hrules=True)
with open(presentation_dir / 'total_accuracy_for_all_validation.tex', 'w') as f:
    f.write(latex)

In [18]:
df_req = dfs_req[index]

In [19]:
n_osw_passes = df_req.surface_water.sum()
n_pws_passes = df_req.partial_surface_water.sum()
n_both_pass = (df_req.surface_water & df_req.partial_surface_water).sum()
n_pws_passes, n_osw_passes, n_both_pass

(44, 44, 41)

In [20]:
n_osw_fails = (~df_req.surface_water).sum()
n_pws_fails = (~df_req.partial_surface_water).sum()
n_both_fail = (~df_req.surface_water | ~df_req.partial_surface_water).sum()
n_osw_fails, n_pws_fails, n_both_pass

(8, 8, 41)

In [21]:
df_passes = pd.DataFrame([{'Class': 'Open Surface Water (OSW)',
                          'Pass': n_osw_passes,
                          'Not Pass': n_osw_fails},
                         {'Class': 'Partial Surface Water (PSW)',
                          'Pass': n_pws_passes,
                          'Not Pass': n_pws_fails},
                         {'Class': 'Both (OSW + PSW)',
                          'Pass': n_both_pass,
                          'Not Pass': n_both_fail}])
df_passes = df_passes.set_index('Class')
df_passes

,Pass,Not Pass
Class,,
Open Surface Water (OSW),44,8
Partial Surface Water (PSW),44,8
Both (OSW + PSW),41,11


In [22]:
latex = df_passes.style.to_latex(multirow_align='t', hrules=True)
with open(presentation_dir / 'total_passes.tex', 'w') as f:
    f.write(latex)